In [42]:
import numpy as np
import time
import os

In [43]:
current_path = os.getcwd()
filename = os.path.join(current_path, "matrix_100x100.csv")
matrix1 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix1),np.array_equal(matrix1, matrix1.T))

filename = os.path.join(current_path, "matrix_200x200.csv")
matrix2 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix2),np.array_equal(matrix2, matrix2.T))

filename = os.path.join(current_path, "matrix_400x400.csv")
matrix4 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix4),np.array_equal(matrix4, matrix4.T))

filename = os.path.join(current_path, "matrix_800x800.csv")
matrix8 = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

(100, 100) True
(200, 200) True
(400, 400) True
(800, 800) True


In [44]:
epsilon = 1e-8  # Using exponentiation
MAX_ITERATIONS = 1000

In [45]:
def norm(v):
    return np.sqrt(np.dot(v,v))

def normalize(v):
    return v/norm(v)

In [46]:
def qr_householder(A):
    m, n = A.shape
    R = A.astype(float)  # Avoid unnecessary copy, ensure mutability
    Q = np.eye(m)

    for i in range(n - 1):
        # Compute the Householder vector
        u = R[i:, i].copy()  # Work with a slice; make a copy to modify
        alpha = norm(u)
        u[0] -= alpha  # Create the Householder vector in place
        norm_u = norm(u)

        if norm_u != 0:  # Avoid division by zero
            u /= norm_u

            # Apply the reflection to R in place
            R[i:, i:] -= 2 * np.outer(u, u @ R[i:, i:])
            
            # Apply the reflection to Q in place
            Q[:, i:] -= 2 * np.outer(Q[:, i:] @ u, u)
    return Q, R


In [47]:
def createRQ(A):
    Q, R = qr_householder(A)
    B = np.matmul(R, Q)
    return B, Q

In [48]:
def qr_algorithm_with_shifts(A, refeig=-1, sindex=-1):
    # Compute the initial QR decomposition of A
    B, Q = createRQ(A)
    
    n = 0  # Iteration counter
    leig = B[refeig, refeig]  # Initial reference eigenvalue
    diff = 1  # Difference tracker for convergence

    # Iterate until the eigenvalues stabilize (very small difference)
    while diff > epsilon and n < MAX_ITERATIONS:
        shift = np.eye(len(B)) * B[sindex, sindex]  # Create shift matrix
        C = B - shift  # Apply shift to B
        B, Q = createRQ(C)  # Perform QR decomposition
        
        for i in range(len(B)): # Undo the shift
            B[i, i] += shift[i, i]  

        n += 1  # Count iterations
        diff = abs(leig - B[refeig, refeig])  # Compute change in eigenvalue
        leig = B[refeig, refeig]  # Update reference eigenvalue

    # Extract eigenvalues (diagonal elements of B)
    eigs = [B[i, i] for i in range(len(B))]
    
    # Extract eigenvectors (columns of Q)
    eigv = [Q[:, i] for i in range(Q.shape[1])]
    return eigs, eigv



In [49]:
matrix = matrix8

In [ ]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts(matrix)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(matrix)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")